This file shows a minimal example how to customize 'train_step' using the functional api of AI4water

In [4]:
# try:
#     import AI4Water
# except ImportError:
#     !python -m pip install "AI4Water[all] @ git+https://github.com/AtrCheema/AI4Water.git"

import site
site.addsitedir("D:\\mytools\\AI4Water")

In [1]:
import tensorflow as tf

tf.__version__

d:\c\anaconda3\envs\tfcpu21_py37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
d:\c\anaconda3\envs\tfcpu21_py37\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
d:\c\anaconda3\envs\tfcpu21_py37\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


'2.1.0'

In [ ]:
assert int(''.join(tf.__version__.split('.')[0:2]).ljust(3, '0')) >= 230, f"""This example is only tested with
        tensorflow versions above 2.3.0. Your version is {tf.__version__}"""

In [ ]:
from ai4water.functional import Model
from ai4water.datasets import arg_beach

In [2]:

class CustomModel(tf.keras.models.Model):

    def train_step(self, data):
        print('custom train_step')
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

In [3]:
# Define Model
layers = {'Dense_0': {'config':  {'units': 64, 'activation': 'relu'}},
          'Dropout_0': {'config':  {'rate': 0.3}},
          'Dense_1': {'config':  {'units': 32, 'activation': 'relu'}},
          'Dropout_1': {'config':  {'rate': 0.3}},
          'Dense_2': {'config':  {'units': 16, 'activation': 'relu'}},
          'Dense_3': {'config':  {'units': 9}},
          'Reshape': {"target_shape": (9, 1)}
         }

In [ ]:
model = Model(
    model={"layers": {"Dense": 8}},
    batch_size=32,
    lookback=1,
    lr=8.95e-5,
    data=arg_beach(),
    epochs=2,
    KModel=CustomModel,
    train_data='random',
)

In [ ]:
history = model.fit()

# since the statement 'custom train_step' is printed, we have verified that tensorflow
# used our own customized train_step during training.